In [1]:
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/18fbeQOzN4BMn09LPnFgWflhTP-r9JJrc?usp=sharing
!pip install -r requirements.txt

Retrieving folder contents
Retrieving folder 1TwkISGfIOUwoK4XmDNyKjpWw4TM7HR3Y converted
Processing file 17HFEkH03lX3hIe_WwpOpVMpQ4m05ZD4b encoder.pkl
Processing file 18hplRkydx9gAaQPnkbqOswUpfgmkqZt3 test_data.pt
Processing file 1dUmFaFHcFJkHK6EHBsmp8kbEr5WlqTGO test_labels.json
Processing file 19sedJZnpR-er_-NyfONMAJzMCvQdSIS9 train_data.pt
Processing file 1lpvMNj1Lx-DloHB2yor96vQZZBBe70pL train_labels.json
Processing file 1ATTeQhrd7xyFz-DKwWF_0Ogmr6GPnG_S unlabeled_data.pt
Processing file 1eRFuUEBdT6mE2yUZ1hTGnTB8ulipU_ef val_data.pt
Processing file 16WtSxNdrjYtoOuvB_OFZkZ9b8g-h-fGz val_labels.json
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=17HFEkH03lX3hIe_WwpOpVMpQ4m05ZD4b
To: /home/dawid/studies/master-1/dl/DeepLearning2024-proj-2/data/converted/encoder.pkl
100%|██████████████████████████████████████████| 580/580 [00:00<00:00, 2.45MB/s]
Downloading...
From (original): 

In [2]:
import pytorch_lightning as pl
import torch

from torch import Tensor, nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks import EarlyStopping

from engine.data import get_data_loader
from engine.metrics import accuracy
from engine.model_base import LightningBaseModule

torch.set_float32_matmul_precision("medium")

/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-13 08:44:45.074078: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 08:44:45.452776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 08:44:46.300635: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not

In [3]:
train_loader = get_data_loader("train")
val_loader = get_data_loader("val")
test_loader = get_data_loader("test")

In [4]:
EMBEDDING_SIZE = 80


class Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.attention = nn.MultiheadAttention(EMBEDDING_SIZE, 10)
        self.activation = nn.ReLU()

    def forward(self, X):
        output, _ = self.attention(X, X, X)
        return self.activation(output)


class AttentionBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.attentions = nn.Sequential(*[Attention() for _ in range(3)])
        self.layer_norm = nn.LayerNorm(EMBEDDING_SIZE)

    def forward(self, X):
        output = self.attentions(X)
        output = self.layer_norm(output)
        return output + X


class AttentionModel(LightningBaseModule):
    def __init__(self):
        super().__init__()
        self.attention_1 = nn.Sequential(*[AttentionBlock() for _ in range(5)])
        self.attention_2 = nn.Sequential(*[AttentionBlock() for _ in range(5)])
        self.flatten = nn.Flatten()
        self.dense = nn.Sequential(
            nn.Linear(8000, 1000),
            nn.ReLU(),
            nn.Linear(1000, 22),
            nn.Softmax(dim=1),
        )
        self.loss = nn.CrossEntropyLoss()

    def forward(self, X):
        output = self.attention_1(X)
        output = self.attention_2(output + X)
        output = self.flatten(output)
        output = self.dense(output)
        return output

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = ReduceLROnPlateau(optimizer, patience=5)
        return [optimizer], [
            {
                "scheduler": scheduler,
                "interval": "epoch",
                "monitor": "val_accuracy",
                "frequency": 1,
            }
        ]

In [5]:
callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        mode="max",
        patience=10,
        min_delta=1e-4,
    )
]

model = AttentionModel().cuda()
trainer = pl.Trainer(max_epochs=100, callbacks=callbacks)
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/dawid/studies/master-1/dl/DeepLearning2024-proj-2/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | attention_1 | Sequential       | 389 K 
1 | attention_2 | Sequential       | 389 K 
2 | flatten     | Flatten          | 0     
3 | dense       | Sequential       | 8.0 M 
4 | loss        | CrossEntropyLoss | 0     
-------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.209    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 5: 100%|██████████| 837/837 [00:40<00:00, 20.82it/s, v_num=0, train_step_loss=2.380, val_step_loss=2.570]

/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
